# Задание

У каждого есть свой набор данный, который можно найти в ведомости на вкладке "Логистическая регрессия":

https://clck.ru/32BVWe

Скачиваете свой датасет с kaggle (для этого нужно авторизоваться с гугл аккаунт и скачать датасет).

**Ваша задача:**
1. Исследовать данные
2. Создать тренировочную и тестовую подвыборки
3. Построить модель логистической регрессии по своему набору данных (т.е. создать модель и обучить ее на тренировочной выборке)
4. Получить коэффиценты уравнения логистической регрессии (т.е. уравнение разделяющей поверхности). Сделать вывод какие из признаков оказались наиболее значимыми, какие нет и почему
5. Написать уравнение логистической регрессии.
6. Вывести confusion matrix для получившейся модели.
7. Вручную посчитать метрики: accuracy, recall, precision. f1.
8. Посчитать метрики с помощью библиотеки sklearn и сравнить с оценками из п.7.
9. Сделать письменный вывод получившихся результатов (хорошие или плохие получлись метрики и почему).

ВСЕ графики должны быть подписанны.
Ваша задача получить метрики как можно лучше.

Импорт некоторых библиотек для начала работы с файлом

In [1]:
from google.colab import files
uploaded = files.upload()

Saving mushrooms.csv to mushrooms.csv


In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

Первое задание

In [72]:
df = pd.read_csv('mushrooms.csv')
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [73]:
print(df.isna().sum())

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64


In [74]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df["class"])

LabelEncoder()

In [75]:
# Мне попался очень плохой DF, поэтому нашёл класс с необходимыми функциями для того, чтобы весь DF сразу привести к нужным значениям
# https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [79]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

df = MultiColumnLabelEncoder(columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']).fit_transform(df)

In [80]:
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


Второе задание

In [81]:
from sklearn.model_selection import train_test_split

X = df.drop(['class'], axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Третье задание

In [82]:
model = LogisticRegression()

model.fit(X_train, y_train)

answers_pred = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Четвёртое задание

In [83]:
model.coef_
#меньше всего влияют fixed acidity, residual sugar и chlorides.

array([[-0.0546573 ,  0.45723512, -0.05763561, -0.38680783, -0.53474215,
        -0.54775309, -7.37634327,  8.02620828, -0.11739435, -0.13417338,
        -2.20352923, -5.0110106 , -0.17595233, -0.15185073, -0.04269061,
         0.        ,  5.26281094,  1.17380496,  1.06069176, -0.22574336,
        -0.41227742,  0.07931462]])

In [84]:
model.intercept_

array([1.40506382])

Пятое задание

y = 0.72483381 + -0.03374035x + 0.46231064x + -0.0559551x + -0.63269276x + -0.52395236x + -0.40422276x + -6.4690726x + 7.35136415x + -0.12364671x + -0.08983065x + -1.79196584x + -4.33715705x + -0.31157805x + -0.14606142x + -0.05478661x + 0x + 5.05660235x + 1.19727022x + 0.88135005x + -0.21933145x + -0.36111057x + 0.06800883x

Шестое задание

In [85]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

confusion_matrix(y_test, answers_pred)

array([[1220,   49],
       [  75, 1094]])

Седьмое задание

In [86]:
tn, fp, fn, tp = confusion_matrix(y_test, answers_pred).ravel()

In [87]:
accuracy = (tp + tn)/(tp + tn + fp + fn)
accuracy

0.9491386382280558

In [88]:
recall = tp/(tp + fn)
recall

0.9358426005132592

In [89]:
precision = tp/(tp + fp)
precision

0.9571303587051618

In [90]:
f1 = 2 * ((precision * recall)/(precision + recall))
f1

0.9463667820069204

Восьмое задание

In [91]:
accuracy_score(y_test, answers_pred)

0.9491386382280558

In [92]:
recall_score(y_test, answers_pred)

0.9358426005132592

In [93]:
precision_score(y_test, answers_pred)

0.9571303587051618

In [94]:
f1_score(y_test, answers_pred)

0.9463667820069204

Девятое задание

In [95]:
# Вывод: метрика Accuracy получилась довольно высокая, следовательно, метрика приближена к идеальной

In [96]:
# Вывод: точность высокая, следовательно меньше ложных срабатываний, тоже самое и с полнотой

In [97]:
# Вывод: обученная модель работает достаточно хорошо